In [3]:
!pip install triton==2.3.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.2 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.4.0
    Uninstalling triton-3.4.0:
      Successfully uninstalled triton-3.4.0


In [1]:
!pip uninstall -y torch torchvision torchaudio transformers numpy
!pip install torch==2.3.1 torchvision torchaudio
!pip install numpy==1.26.4
!pip install bitsandbytes==0.43.3 accelerate==0.33.0 transformers==4.45.1
!pip install langchain langchain_community sentence-transformers faiss-cpu PyPDF2 python-dotenv --quiet
!pip install pypdf


Found existing installation: torch 2.3.1
Uninstalling torch-2.3.1:
  Successfully uninstalled torch-2.3.1
Found existing installation: torchvision 0.18.1
Uninstalling torchvision-0.18.1:
  Successfully uninstalled torchvision-0.18.1
Found existing installation: torchaudio 2.3.1
Uninstalling torchaudio-2.3.1:
  Successfully uninstalled torchaudio-2.3.1
Found existing installation: transformers 4.45.1
Uninstalling transformers-4.45.1:
  Successfully uninstalled transformers-4.45.1
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached torch-2.3.1-cp312-cp312-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchvision-0.24.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (5.9 kB)
  Using cached torchaudio-2.9.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.9 kB)
  Using cached numpy-2.3.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
INFO: pip is looking at multiple versions of to

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.4
    Uninstalling numpy-2.3.4:
      Successfully uninstalled numpy-2.3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.17.1 requires transformers, which is not installed.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, which is not installed.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incomp

  Using cached transformers-4.45.1-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.45.1-py3-none-any.whl (9.9 MB)


In [2]:
# ============================================
# 🔧 INSTALL
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain.llms import HuggingFacePipeline

# ============================================
# 📚 IMPORTS
# ============================================
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, BitsAndBytesConfig
from google.colab import files

# ============================================
# ⚙️ SETTINGS
# ============================================
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_HUGGINGFACE_TOKEN"  # optional if public
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# ============================================
# 🧠 LOAD 4-BIT QUANTIZED MODEL
# ============================================
print("\n🤖 Loading quantized Mistral-7B (4-bit)... this takes a few minutes...")
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.3,
    do_sample=True
)

llm = HuggingFacePipeline(pipeline=pipe)
print("✅ Quantized Mistral loaded successfully!")

print("✅ Quantized model loaded successfully!")

# ============================================
# 🧠 PROMPT TEMPLATE
# ============================================
prompt_template = """
You are a helpful and intelligent AI assistant.
Use the provided context from the PDF to answer the user's question.
If the user asks for a summary, summarize clearly.
If they ask for explanation, explain simply.
If the answer isn't in the PDF, respond with:
"I couldn't find information about that in this document."

Context:
{context}

Question:
{question}

Answer:
"""

QA_PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_PROMPT)

# ============================================
# 📂 UPLOAD PDF
# ============================================
print("\n📂 Upload your PDF file:")
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]
print(f"✅ Uploaded: {pdf_path}")

# ============================================
# 🧩 LOAD + SPLIT + EMBED
# ============================================
loader = PyPDFLoader(pdf_path)
pages = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(pages)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

print("✅ PDF processed successfully!")

# ============================================
# 💬 CHAT LOOP
# ============================================
while True:
    query = input("\n💭 Ask a question about your PDF (or type 'exit'): ")
    if query.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    docs = retriever.get_relevant_documents(query)
    answer = chain.run(input_documents=docs, question=query)

    print("\n🧾 Answer:")
    print(answer)



🤖 Loading quantized Mistral-7B (4-bit)... this takes a few minutes...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Quantized Mistral loaded successfully!
✅ Quantized model loaded successfully!

📂 Upload your PDF file:


Saving Deep+Learning+Ian+Goodfellow.pdf to Deep+Learning+Ian+Goodfellow.pdf
✅ Uploaded: Deep+Learning+Ian+Goodfellow.pdf


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...


🧾 Answer:

You are a helpful and intelligent AI assistant.
Use the provided context from the PDF to answer the user's question.
If the user asks for a summary, summarize clearly.
If they ask for explanation, explain simply.
If the answer isn't in the PDF, respond with:
"I couldn't find information about that in this document."

Context:
CHAPTER 6. DEEP FEEDFORWARD NETWORKS
6.5 Back-Propagation and Other Diﬀerentiation Algo-
rithms
When we use a feedforward neural network to accept an input x and produce an
output ˆy, information ﬂows forward through the network. The inputs x provide
the initial information that then propagates up to the hidden units at each layer
and ﬁnally produces ˆy . This is called forwardpropagation. During training,
forward propagation can continue onward until it produces a scalar cost J(θ).
The back-propagationalgorithm ( , ), often simply calledRumelhart et al. 1986a
backprop, allows the information from the cost to then ﬂow backwards through
the network, in 

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



🧾 Answer:

You are a helpful and intelligent AI assistant.
Use the provided context from the PDF to answer the user's question.
If the user asks for a summary, summarize clearly.
If they ask for explanation, explain simply.
If the answer isn't in the PDF, respond with:
"I couldn't find information about that in this document."

Context:
deep tree, rather than the chain-like structure of RNNs. The typical computational
graph for a recursive network is illustrated in ﬁgure . Recursive neural10.14
2 We suggest to not abbreviate “recursive neural network” as “RNN” to avoid confusion with
“recurrent neural network.”
400

graph interpretation of the RNN is based on ignoring the hidden units h( )t by
marginalizing them out of the model.
It is more interesting to consider the graphical model structure of RNNs that
results from regarding the hidden units h( )t as random variables.1 Including the
hidden units in the graphical model reveals that the RNN provides a very eﬃcient
parametrization of

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



🧾 Answer:

You are a helpful and intelligent AI assistant.
Use the provided context from the PDF to answer the user's question.
If the user asks for a summary, summarize clearly.
If they ask for explanation, explain simply.
If the answer isn't in the PDF, respond with:
"I couldn't find information about that in this document."

Context:
or statistics background, but want to rapidly acquire one and begin using deep
learning in their product or platform. Deep learning has already proven useful in
8

data has increased.
• Deep learning models have grown in size over time as computer infrastructure
(both hardware and software) for deep learning has improved.
• Deep learninghas solvedincreasingly complicated applicationswith increasing
accuracy over time.
11

Deep learning allows the computer to build complex concepts out of simpler con-
cepts. Figure shows how a deep learning system can represent the concept of1.2
an image of a person by combining simpler concepts, such as corners and con